In [1]:
! pip install cma

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import timeit

import numpy as np
import cma
from IPython.display import display, clear_output

from anguilla.optimizers.cma import CMA, StrategyParameters, StoppingConditions
from anguilla.fitness.benchmark import Sphere, SumSquares

In [4]:
def test_with_pycma(n=4, d=3):
    fitness_function = Sphere(d)
    initial_point = fitness_function.random_points(1)
    initial_sigma = 0.3
    cma.fmin(fitness_function, initial_point, initial_sigma, options={'CMA_Active': 0})

test_with_pycma()

(3_w,7)-CMA-ES (mu_w=2.3,w_1=58%) in dimension 3 (seed=84634, Thu Dec  3 17:05:57 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      7 1.830289009606220e-01 1.0e+00 3.00e-01  3e-01  3e-01 0:00.0
    2     14 5.129514262188250e-02 1.2e+00 3.36e-01  3e-01  4e-01 0:00.0
    3     21 3.717807645583373e-01 1.4e+00 3.37e-01  3e-01  4e-01 0:00.0
   96    672 5.823227703555781e-15 1.6e+00 9.70e-06  7e-08  1e-07 0:00.3
termination on tolfun=1e-11 (Thu Dec  3 17:06:01 2020)
final/bestever f-value = 3.069588e-15 3.069588e-15
incumbent solution: [8.463640111882815e-09, -9.212800156691295e-09, -5.397294361878817e-08]
std deviation: [1.044696143369427e-07, 7.080393757770522e-08, 7.965942174222113e-08]


In [5]:
def test_with_prototype(n=4,d=3):
    parameters = StrategyParameters(n=n)
    conditions = StoppingConditions(n=n,
                                    population_size=parameters.population_size,
                                    ftarget=1e-14,
                                    )
    fitness_function = Sphere(d)
    initial_point = fitness_function.random_points(1)
    optimizer = CMA(initial_point=initial_point,
                    initial_sigma=0.3,
                    strategy_parameters=parameters,
                    stopping_conditions=conditions)
    while not optimizer.stop():
        candidate_solutions = optimizer.ask()
        function_values = fitness_function(candidate_solutions.T)
        function_evaluations = fitness_function.evaluation_count
        ranked_indices = optimizer.rank(candidate_solutions, function_values, function_evaluations)
        optimizer.tell(candidate_solutions, ranked_indices)
        if (optimizer._fevals % 20) == 0:
            display(optimizer._fevals, optimizer._best_value, optimizer._best_solution)
    display(optimizer._fevals, optimizer._best_value, optimizer._best_solution)

test_with_prototype(n=4)

20

1.2632946512518723

array([-0.0034661 , -0.99632687, -0.52020707])

40

0.10572180857134589

...

1.1333508882191525e-14

array([-6.54382514e-08, -1.66907459e-08, -8.22967991e-08])

610

1.468215888239096e-15

array([-1.43990795e-08,  3.51280064e-08, -5.18705754e-09])